# GBDT学习目标
- 掌握提升书的算法原理和思想
- 了解梯度提升树的原理思想

--- 


# 1、什么是提升树？
概念：提升树（Boosting Tree）是一种集成学习算法，它通过组合多个弱学习器（通常是决策树）来构建一个强学习器
核心思想：通过顺序的添加弱学习器，每个新的弱学习器都尝试修正前一个模型错误，从而逐步提升整体模型的性能

# 2、什么是梯度提升树？
概念：梯度提升树（Gradient Boosting）是提升树（Boosting Tree）的一种改进算法，`梯度提升树不再使用拟合残差，而是利用最速下降的近似方法，利用损失函数的负梯度作为提升树算法中的残差近似值`。

# 3、什么是初始化弱学习器（CART数），如何构建一个初始化弱学习器？
概念：CART树全称分类与回归树（Classification and Regression Trees）是一种常用的决策树算法，它通过递归地将数据分割成更小的子集，每个字节包含更小似的数据点，从而构建一个树结构的决策模型。
核心思想：利用特征对数据进行分割，使每个分割后子集都尽可能具有一致性（即分类问题中的类别一致性和回归问题中的数值一致性）
构建步骤：
- 选择最佳分割特征和分割点：通过计算每个特征的每个可能的分割点，选择使得数据集分割后，子集内部一致性最高的特征和分割点。
- 递归分割：对分割后的每个子集重复上述分割过程，直到满足停止条件（如子集中的样本数小于某个阈值，或达到预设的树深度）。
- 构建树模型：最终得到的树状结构即为决策树模型，每个节点代表一个特征的分割，每个叶节点代表一个决策结果。

# 4、GBDT算法原理是什么？
概念：GBDT算法是指梯度提升决策树算法（Gradient Boosting Decision Trees），是一种集成学习方法，主要用于解决回归、分类和排序问题
特征：
- GBDT通过迭代地训练决策树来最小化损失函数，每次迭代都在前一次的基础上改进模型
- GBDT在许多机器学习竞赛和实际应用中都表现出色，因为它能够处理各种类型的数据，并且对特征的缺失和异常值具有较好的鲁棒性

核心思想：构建多个弱学习器（通常是决策树），然后将它们组合起来形成一个强学习器


# 5、如何增加个学习器的多样性，有哪些常见的方法？
通常为了增加个体学习器的多样性，在学习过程中引入随机行，一般是增加以下扰动来增加样本的多样性：
- 输入属性的扰动：
- 算法参数的扰动
- 数据样本的扰动


# 总结
- 提升树中每一个弱学习器都可以通过拟合残差来构建强学习器
- 梯度提升树中的每一个弱学习器通过你和负梯度构建强学习器


In [4]:
# 导入Pandas库（主要在数据分析和机器学习中，Pandas常用数据清洗、数据分析、数据探索）
import pandas as pd


# 1、数据加载
titanic = pd.read_csv('../file/titanic.csv')
titanic.info()
'''
    数据条目：891
    字段数量和类型：15 布尔类型、64位浮点数、64位整型、对象类型
    为空字段：age、embarked、deck、embark_town
    占用内存： 92.5 KB
'''


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


'\n    数据条目：891\n    字段数量和类型：15 布尔类型、64位浮点数、64位整型、对象类型\n    为空字段：age、embarked、deck、embark_town\n    占用内存： 92.5 KB\n'

In [26]:
# 2、特征工程
X = titanic[['pclass', 'age', 'sex']]
y = titanic['survived']
print(f'特征工程填充前：{X.info()}')
X['age'].fillna(X['age'].mean(), inplace = True)
print(f'特征工程填充后：{X.info()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  891 non-null    int64  
 1   age     714 non-null    float64
 2   sex     891 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB
特征工程填充前：None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  891 non-null    int64  
 1   age     891 non-null    float64
 2   sex     891 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB
特征工程填充后：None


/tmp/ipykernel_12278/817711746.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(X['age'].mean(), inplace = True)


In [29]:
# 导入sklearn.model_selection测试集训练集划分包 （model_selection包含数据分割和交叉验证包）
from sklearn.model_selection import train_test_split
# 导入sklearn.feature_extraction 文本数据中提取特征的工具
from sklearn.feature_extraction import DictVectorizer

# 3、切分数据以及特征处理
# 测试集和训练集划分
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 22)
'''
    Description：数据格式转化为特征向量
    Param:
        - sparse：True：密集矩阵 False：稀疏矩阵
''' 
vector = DictVectorizer(sparse = False)
x_train = vector.fit_transform(x_train.to_dict(orient = 'records'))
x_test = vector.fit_transform(x_test.to_dict(orient = 'records'))
print(f'x_train矩阵：{x_train[:2]}')
print(f'x_test矩阵：{x_test[:2]}')
print(f'矩阵属性：{vector.feature_names_}')

x_train矩阵：[[28.   2.   1.   0. ]
 [45.5  1.   0.   1. ]]
x_test矩阵：[[23.  3.  1.  0.]
 [46.  1.  0.  1.]]
矩阵属性：['age', 'pclass', 'sex=female', 'sex=male']


In [23]:
# DictVectorizer方法使用Demo
from sklearn.feature_extraction import DictVectorizer

# 创建字典数组作为输入数据（每个字典对象都会创建一个矩阵，不存在的属性会被填充成0）
data = [
    {'color': 'red', 'size': 'big'},
    {'color': 'blue', 'size': 'small'},
    {'color': 'green', 'size': 'medium'}
]

# 创建DictVectorizer对象
vectorizer = DictVectorizer(sparse = False)

# 转换数据集
X = vectorizer.fit_transform(data)

# 打印特征矩阵和特征名
print('X:', X)
# 获取对象属性
# print('vectorizer:', dir(vectorizer))
print('vectorizer:', vectorizer.feature_names_)


X: [[0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0.]]
vectorizer: ['color=blue', 'color=green', 'color=red', 'size=big', 'size=medium', 'size=small']


In [34]:
# 导入sklearn.tree模块中决策树模型（tree模块包含：决策树分类器、回归树、随机森林）
from sklearn.tree import DecisionTreeClassifier
# 导入sklearn.ensemble模块中随机森林模型（ensemble模块包含：Bagging、RandomForest、AdaBoost、GradientBoosting（GradientBoosting中包含：GBDT和XGBoost））
from sklearn.ensemble import RandomForestClassifier
# 导入sklearn.ensemble中GBDT模型（梯度提升树）
from sklearn.ensemble import GradientBoostingClassifier

# 4、使用三种分类器训练以及预测

dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)
dtc_y_predict = dtc.predict(x_test)
print(f'决策树模型得分：{dtc.score(x_test, y_test)}')

rfc = RandomForestClassifier(random_state = 9)
rfc.fit(x_train, y_train)
rfc_y_predict = rfc.predict(x_test)
print(f'随机森林模型得分：{rfc.score(x_test, y_test)}')

gbc = GradientBoostingClassifier()
gbc.fit(x_train, y_train)
gbc_y_predict = gbc.predict(x_test)
print(f'GBDT模型得分：{gbc.score(x_test, y_test)}')

决策树模型得分：0.7802690582959642
随机森林模型得分：0.7802690582959642
GBDT模型得分：0.7623318385650224


In [37]:
# 导入sklearn.metrics中分类评估模型（其中包含分类指标、精确度、召回率、F1分出）
from sklearn.metrics import classification_report
# 5、性能评估
dtc_report = classification_report(dtc_y_predict, y_test)
print(f'决策树模型性能报告：{dtc_report}')

rfc_report = classification_report(rfc_y_predict, y_test)
print(f'随机森林模型性能报告：{rfc_report}')

gbc_report = classification_report(dtc_y_predict, y_test)
print(f'GBDT模型性能报告：{gbc_report}')

决策树模型性能报告：              precision    recall  f1-score   support

           0       0.86      0.79      0.82       146
           1       0.66      0.77      0.71        77

    accuracy                           0.78       223
   macro avg       0.76      0.78      0.77       223
weighted avg       0.79      0.78      0.78       223

随机森林模型性能报告：              precision    recall  f1-score   support

           0       0.83      0.80      0.82       138
           1       0.70      0.74      0.72        85

    accuracy                           0.78       223
   macro avg       0.77      0.77      0.77       223
weighted avg       0.78      0.78      0.78       223

GBDT模型性能报告：              precision    recall  f1-score   support

           0       0.86      0.79      0.82       146
           1       0.66      0.77      0.71        77

    accuracy                           0.78       223
   macro avg       0.76      0.78      0.77       223
weighted avg       0.79      0.78      0.7